# Preparing Data For Mahine Learning Models

## In this Notebook, we will process the train and the test data in different ways to prepare for modeling

In [ ]:
# Importing relevant libraries
from bs4 import BeautifulSoup as soup
import requests
import re
import boto3
import sys
import os
import pandas as pd
import csv
import s3fs
from math import pi

from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, StructType
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, slice, count, when, expr, isnan, isnull, min, max, avg, sin, log10, cos 
from pyspark.sql.functions import date_format, to_timestamp, concat, unix_timestamp, substring, lit
from pyspark.sql.functions import col, month, quarter, dayofweek,dayofmonth, year, dayofyear
from pyspark.sql import functions as f
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.window import Window


#model
from pyspark.ml.regression import RandomForestRegressor, LinearRegression
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.stat import Correlation
from pyspark_dist_explore import hist

import configparser
import findspark
import lxml
from datetime import timedelta
from pandas.tseries.offsets import BDay
import itertools
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
### Starting Pyspark Session


spark = SparkSession.builder\
                    .config('spark.master','local[*]')\
                    .config('spark.add.name','S3app')\
                    .config('spark.jars.packages','org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-common:3.3.4')\
                    .config("spark.driver.memory", "16g") \
                    .getOrCreate()


In [4]:
### Configuring Pyspark to read data from S3 Bucket. 

findspark.init()
config = configparser.ConfigParser()
# AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
# AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

aws_profile = 'default'
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get(aws_profile, "aws_access_key_id") 
access_key = config.get(aws_profile, "aws_secret_access_key")
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# spark.conf.set("spark.network.io.preferDirectBufs", "false")

# spark.conf.set("spark.sql.adaptive.enabled", "true")
# spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.committer.name","magic")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.awsAccessKeyId", access_id)
hadoop_conf.set("fs.s3a.awsSecretAccessKey", access_key)
# hadoop_conf.set('spark.sql.files.maxPartitionBytes','134217728')
# hadoop_conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# hadoop_conf.set("spark.sql.autoBroadcastJoinThreshold", '104857600')
# hadoop_conf.set("spark.sql.autoBroadcastJoinThreshold", '-1')

# Processing the train and the test data

## Processing data with no conversion to unit circle for the cyclical columns 

In [214]:
# Reading the data
df = spark.read.parquet("s3a://w210-bucket/data_wrangling/final_df_prior_hr.parquet")


# casting certain columns to integer
df = df.withColumn('day_of_month', dayofmonth(df.date)) \
       .withColumn('day_of_year', dayofyear(df.date)) 
# df = df.withColumn('number_of_searches',col('number_of_searches').cast('integer'))
df= df.withColumn("year",col("year").cast('integer'))  
      # .withColumn("year",col("year").cast('integer')) 
#       .withColumn("month",col("month").cast('integer')) \
#       .withColumn("quarter",col("quarter").cast('integer')) \
#       .withColumn("day_of_week",col("day_of_week").cast('integer')) \
#       .withColumn("day_of_month",col("day_of_month").cast('integer')) \
      # .withColumn("holiday_period",col("holiday_period").cast('integer')) \
      # .withColumn("event_origin",col("event_origin").cast('integer')) \
      # .withColumn("event_dest",col("event_dest").cast('integer'))
    

# mapping numbers to weather types for origin
df = df.withColumn("wth_type_origin_cat", when(df.wth_type_origin == "no_precipitation",0)
                                 .when(df.wth_type_origin == 'Rain',1)
                                 .when(df.wth_type_origin == 'hail',2)
                                 .when(df.wth_type_origin == 'Drizzle',3)
                                 .when(df.wth_type_origin == 'Snow',4))\
        .withColumn("wth_type_origin_cat",col("wth_type_origin_cat").cast('string'))

# mapping numbers to weather types for destination
df = df.withColumn("wth_type_dest_cat", when(df.wth_type_dest == "no_precipitation",0)
                                 .when(df.wth_type_dest == 'Rain',1)
                                 .when(df.wth_type_dest == 'hail',2)
                                 .when(df.wth_type_dest == 'Drizzle',3)
                                 .when(df.wth_type_dest == 'Snow',4))  \
       .withColumn("wth_type_dest_cat",col("wth_type_dest_cat").cast('string'))

In [215]:
# zeroing out the null rows for average_route_crowd and prior_hr_route_crowd columns
df = df.withColumn('average_route_crowd',f.when(f.col("average_route_crowd").isNull(),f.lit(0)).otherwise(df.average_route_crowd))
df = df.withColumn('prior_hr_route_crowd',f.when(f.col("prior_hr_route_crowd").isNull(),f.lit(0)).otherwise(df.prior_hr_route_crowd))

In [218]:
# categorical feature list
columns_categorical = ['hour','day_of_week','month','day_of_month','quarter','day_of_year','origin','destination','event_origin','event_dest',
                       'wth_type_origin_cat','wth_type_dest_cat','holiday_period']

# continous feature list
columns_continues = ['year','wind_speed_origin','air_temp_origin','precipitation_origin',
                     'wind_speed_dest','air_temp_dest','precipitation_dest','number_of_searches','average_route_crowd','prior_hr_route_crowd']

##### indexing and one hot encoding categorical features

indexer = StringIndexer(inputCol="hour", outputCol="hour_index")
df_final = indexer.fit(df).transform(df)
ohe = OneHotEncoder(inputCol="hour_index", outputCol="hour_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="day_of_week", outputCol="day_of_week_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="day_of_week_index", outputCol="day_of_week_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="month", outputCol="month_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="month_index", outputCol="month_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="day_of_month", outputCol="day_of_month_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="day_of_month_index", outputCol="day_of_month_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="quarter", outputCol="quarter_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="quarter_index", outputCol="quarter_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="day_of_year", outputCol="day_of_year_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="day_of_year_index", outputCol="day_of_year_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="origin", outputCol="origin_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="origin_index", outputCol="origin_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="destination", outputCol="destination_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="destination_index", outputCol="destination_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="event_origin", outputCol="event_origin_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="event_origin_index", outputCol="event_origin_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="event_dest", outputCol="event_dest_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="event_dest_index", outputCol="event_dest_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="wth_type_origin_cat", outputCol="wth_type_origin_cat_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="wth_type_origin_cat_index", outputCol="wth_type_origin_cat_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="wth_type_dest_cat", outputCol="wth_type_dest_cat_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="wth_type_dest_cat_index", outputCol="wth_type_dest_cat_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="holiday_period", outputCol="holiday_period_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="holiday_period_index", outputCol="holiday_period_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)


# Vectore assembling categorical features
assembler_categ = VectorAssembler(inputCols= ['hour_index_ohe','day_of_week_index_ohe','month_index_ohe',
                                              'day_of_month_index_ohe','quarter_index_ohe','day_of_year_index_ohe',
                                              'origin_index_ohe','destination_index_ohe','event_origin_index_ohe',
                                              'event_dest_index_ohe','wth_type_origin_cat_index_ohe','wth_type_dest_cat_index_ohe',
                                              'holiday_period_index_ohe'],
                           outputCol="cat_features")


# Vector assembling continues features
assembler_cont = VectorAssembler(inputCols=columns_continues,
                                   outputCol="cont_features")

# transforming the data
df_final = assembler_categ.transform(df_final)
df_final = assembler_cont.transform(df_final)
assembler = VectorAssembler(inputCols= ["cont_features","cat_features"],
                               outputCol="features")

df_final = assembler.transform(df_final)
 

In [219]:
# filtering out 2011 data. Using 2012-2021 as train data and 2022 as a test data
df_final_1 = df_final.filter(col('year') != 2011)
processed_train_df = df_final_1.filter(col('year') != 2022).select('ridership_number','features')
processed_test_df =  df_final_1.filter(col('year') == 2022).select('ridership_number','features')

In [220]:
# # saving as parquet in S3 bucket
# processed_train_df.write.parquet('s3a://w210-bucket/data_wrangling/processed_train_df_1.parquet',mode='overwrite')
# processed_test_df.write.parquet('s3a://w210-bucket/data_wrangling/processed_test_df_1.parquet',mode='overwrite')

## Processing data with conversion to unit circle for the cyclical columns 

In [51]:
# reading the data and extracting new time feature from existing column
df = spark.read.parquet("s3a://w210-bucket/data_wrangling/final_df_prior_hr.parquet")
df = df.withColumn('day_of_month', dayofmonth(df.date)) \
       .withColumn('day_of_year', dayofyear(df.date)) 

In [52]:
# converting cyclical features to unit circles

df = df.withColumn('month_sine', sin((col('month')-1)*2.*pi/12))\
         .withColumn('month_cosine', cos((col('month')-1)*2.*pi/12)) \
         .withColumn('hr_sine', sin(col('hour')*2.*pi/24))\
         .withColumn('hr_cosine', cos(col('hour')*2.*pi/24)) \
         .withColumn('day_of_week_sine', sin((col('day_of_week')-1)*2.*pi/7))\
         .withColumn('day_of_week_cosine', cos((col('day_of_week')-1)*2.*pi/7)) \
         .withColumn('quarter_sine', sin((col('quarter')-1)*2.*pi/4))\
         .withColumn('quarter_cosine', cos((col('quarter')-1)*2.*pi/4)) \
         .withColumn("year",col("year").cast('integer')) 
        

In [53]:
# mapping numbers to weather types for origin
df = df.withColumn("wth_type_origin_cat", when(df.wth_type_origin == "no_precipitation",0)
                                 .when(df.wth_type_origin == 'Rain',1)
                                 .when(df.wth_type_origin == 'hail',2)
                                 .when(df.wth_type_origin == 'Drizzle',3)
                                 .when(df.wth_type_origin == 'Snow',4))\
        .withColumn("wth_type_origin_cat",col("wth_type_origin_cat").cast('string'))

# mapping numbers to weather types for destination
df = df.withColumn("wth_type_dest_cat", when(df.wth_type_dest == "no_precipitation",0)
                                 .when(df.wth_type_dest == 'Rain',1)
                                 .when(df.wth_type_dest == 'hail',2)
                                 .when(df.wth_type_dest == 'Drizzle',3)
                                 .when(df.wth_type_dest == 'Snow',4))  \
       .withColumn("wth_type_dest_cat",col("wth_type_dest_cat").cast('string'))

In [54]:
# zeroing out the null rows for average_route_crowd and prior_hr_route_crowd columns
df = df.withColumn('average_route_crowd',f.when(f.col("average_route_crowd").isNull(),f.lit(0)).otherwise(df.average_route_crowd))
df = df.withColumn('prior_hr_route_crowd',f.when(f.col("prior_hr_route_crowd").isNull(),f.lit(0)).otherwise(df.prior_hr_route_crowd))

In [47]:
# np.sin(2*(2.*np.pi/4))
# np.cos(2*(2.*np.pi/4))

In [56]:
# list of categorical features
columns_categorical = ['day_of_month','day_of_year','origin','destination','event_origin','event_dest',
                       'wth_type_origin_cat','wth_type_dest_cat','holiday_period']


# list of continous features
columns_continues = ['month_sine','month_cosine','hr_sine','hr_cosine','day_of_week_sine','day_of_week_cosine','quarter_sine','quarter_cosine',
                     'wind_speed_origin','air_temp_origin','precipitation_origin',
                     'wind_speed_dest','air_temp_dest','precipitation_dest','number_of_searches','average_route_crowd','prior_hr_route_crowd']

#### indexing and one encoding the categorical features
indexer = StringIndexer(inputCol="day_of_month", outputCol="day_of_month_index")
df_final = indexer.fit(df).transform(df)
ohe = OneHotEncoder(inputCol="day_of_month_index", outputCol="day_of_month_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="day_of_year", outputCol="day_of_year_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="day_of_year_index", outputCol="day_of_year_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="origin", outputCol="origin_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="origin_index", outputCol="origin_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="destination", outputCol="destination_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="destination_index", outputCol="destination_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="event_origin", outputCol="event_origin_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="event_origin_index", outputCol="event_origin_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="event_dest", outputCol="event_dest_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="event_dest_index", outputCol="event_dest_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="wth_type_origin_cat", outputCol="wth_type_origin_cat_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="wth_type_origin_cat_index", outputCol="wth_type_origin_cat_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="wth_type_dest_cat", outputCol="wth_type_dest_cat_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="wth_type_dest_cat_index", outputCol="wth_type_dest_cat_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)

indexer = StringIndexer(inputCol="holiday_period", outputCol="holiday_period_index")
df_final = indexer.fit(df_final).transform(df_final)
ohe = OneHotEncoder(inputCol="holiday_period_index", outputCol="holiday_period_index_ohe")
df_final = ohe.fit(df_final).transform(df_final)


# Vector assembling the categorical features
assembler_categ = VectorAssembler(inputCols= ['day_of_month_index_ohe','day_of_year_index_ohe',
                                              'origin_index_ohe','destination_index_ohe','event_origin_index_ohe',
                                              'event_dest_index_ohe','wth_type_origin_cat_index_ohe','wth_type_dest_cat_index_ohe',
                                              'holiday_period_index_ohe'],
                           outputCol="cat_features")


# vector assembling continues variables
assembler_cont = VectorAssembler(inputCols=columns_continues,
                                   outputCol="cont_features")


# transforming the data
df_final = assembler_categ.transform(df_final)
df_final = assembler_cont.transform(df_final)
assembler = VectorAssembler(inputCols= ["cont_features","cat_features"],
                               outputCol="features")

df_final = assembler.transform(df_final)



 

In [57]:
# filtering out 2011 data. Using 2012-2021 as train data and 2022 as a test data
df_final_1 = df_final.filter(col('year') != 2011)
processed_train_df = df_final_1.filter(col('year') != 2022).select('ridership_number','features')
processed_test_df =  df_final_1.filter(col('year') == 2022).select('ridership_number','features')

In [58]:
# # saving as parquet in S3 bucket
# processed_train_df.write.parquet('s3a://w210-bucket/data_wrangling/processed_train_take_6.parquet',mode='overwrite')
# processed_test_df.write.parquet('s3a://w210-bucket/data_wrangling/processed_test_take_6.parquet',mode='overwrite')

In [65]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'log_ridership_number')
dt_model = dt.fit(processed_train_df)
dt_predictions = dt_model.transform(processed_test_df)


23/03/26 00:34:07 WARN MemoryStore: Not enough space to cache rdd_254_6 in memory! (computed 923.4 MiB so far)
23/03/26 00:34:07 WARN BlockManager: Persisting block rdd_254_6 to disk instead.
23/03/26 00:34:08 WARN MemoryStore: Not enough space to cache rdd_254_4 in memory! (computed 923.4 MiB so far)
23/03/26 00:34:08 WARN BlockManager: Persisting block rdd_254_4 to disk instead.
23/03/26 00:34:08 WARN MemoryStore: Not enough space to cache rdd_254_3 in memory! (computed 923.4 MiB so far)
23/03/26 00:34:08 WARN BlockManager: Persisting block rdd_254_3 to disk instead.
23/03/26 00:34:13 WARN MemoryStore: Not enough space to cache rdd_254_7 in memory! (computed 1385.1 MiB so far)
23/03/26 00:34:13 WARN BlockManager: Persisting block rdd_254_7 to disk instead.
23/03/26 00:34:13 WARN MemoryStore: Not enough space to cache rdd_254_1 in memory! (computed 1385.1 MiB so far)
23/03/26 00:34:13 WARN BlockManager: Persisting block rdd_254_1 to disk instead.
23/03/26 00:34:32 WARN MemoryStore: No

23/03/26 00:36:45 WARN MemoryStore: Not enough space to cache rdd_254_7 in memory! (computed 2.1 GiB so far)
23/03/26 00:36:45 WARN MemoryStore: Not enough space to cache rdd_254_2 in memory! (computed 115.2 MiB so far)
23/03/26 00:36:45 WARN MemoryStore: Not enough space to cache rdd_254_6 in memory! (computed 933.5 MiB so far)
23/03/26 00:36:46 WARN MemoryStore: Not enough space to cache rdd_254_1 in memory! (computed 1400.3 MiB so far)
23/03/26 00:36:46 WARN MemoryStore: Not enough space to cache rdd_254_3 in memory! (computed 115.2 MiB so far)
23/03/26 00:36:47 WARN MemoryStore: Not enough space to cache rdd_254_4 in memory! (computed 2.1 GiB so far)
23/03/26 00:36:48 WARN MemoryStore: Not enough space to cache rdd_254_0 in memory! (computed 2.1 GiB so far)


23/03/26 00:37:15 WARN MemoryStore: Not enough space to cache rdd_254_5 in memory! (computed 7.1 GiB so far)


23/03/26 00:37:55 WARN MemoryStore: Not enough space to cache rdd_254_8 in memory! (computed 1385.1 MiB so far)
23/03/26 00:37:55 WARN BlockManager: Persisting block rdd_254_8 to disk instead.
23/03/26 00:37:56 WARN MemoryStore: Not enough space to cache rdd_254_13 in memory! (computed 923.4 MiB so far)
23/03/26 00:37:56 WARN BlockManager: Persisting block rdd_254_13 to disk instead.
23/03/26 00:37:57 WARN MemoryStore: Not enough space to cache rdd_254_9 in memory! (computed 1385.1 MiB so far)
23/03/26 00:37:57 WARN BlockManager: Persisting block rdd_254_9 to disk instead.
23/03/26 00:37:59 WARN MemoryStore: Not enough space to cache rdd_254_10 in memory! (computed 1385.1 MiB so far)
23/03/26 00:37:59 WARN BlockManager: Persisting block rdd_254_10 to disk instead.
23/03/26 00:37:59 WARN MemoryStore: Not enough space to cache rdd_254_15 in memory! (computed 923.4 MiB so far)
23/03/26 00:37:59 WARN BlockManager: Persisting block rdd_254_15 to disk instead.
23/03/26 00:37:59 WARN MemorySt

23/03/26 00:39:19 WARN MemoryStore: Not enough space to cache rdd_254_14 in memory! (computed 3.6 MiB so far)
23/03/26 00:39:20 WARN MemoryStore: Not enough space to cache rdd_254_13 in memory! (computed 1400.3 MiB so far)
23/03/26 00:39:37 WARN MemoryStore: Not enough space to cache rdd_254_16 in memory! (computed 3.0 GiB so far)
23/03/26 00:39:37 WARN BlockManager: Persisting block rdd_254_16 to disk instead.


...

23/03/26 00:50:58 WARN MemoryStore: Not enough space to cache rdd_254_8 in memory! (computed 933.5 MiB so far)
23/03/26 00:50:58 WARN MemoryStore: Not enough space to cache rdd_254_10 in memory! (computed 604.7 MiB so far)
23/03/26 00:50:58 WARN MemoryStore: Not enough space to cache rdd_254_11 in memory! (computed 604.7 MiB so far)
23/03/26 00:50:58 WARN MemoryStore: Not enough space to cache rdd_254_14 in memory! (computed 75.3 MiB so far)
23/03/26 00:50:58 WARN MemoryStore: Not enough space to cache rdd_254_13 in memory! (computed 390.0 MiB so far)


23/03/26 00:51:03 WARN MemoryStore: Not enough space to cache rdd_254_15 in memory! (computed 933.5 MiB so far)
23/03/26 00:51:04 WARN MemoryStore: Not enough space to cache rdd_254_9 in memory! (computed 2.1 GiB so far)


23/03/26 00:51:19 WARN MemoryStore: Not enough space to cache rdd_254_16 in memory! (computed 3.1 GiB so far)


23/03/26 00:51:36 WARN MemoryStore: Not enough space to cache rdd_254_17 in memory! (computed 1400.3 MiB so far)
23/03/26 00:51:37 WARN MemoryStore: Not enough space to cache rdd_254_18 in memory! (computed 1400.3 MiB so far)


23/03/26 00:52:02 WARN MemoryStore: Not enough space to cache rdd_254_19 in memory! (computed 3.1 GiB so far)


In [66]:
dt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)

In [67]:
dt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2 = dt_evaluator.evaluate(dt_predictions)

In [68]:
dt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae = dt_evaluator.evaluate(dt_predictions)

In [69]:
print(f"RMSE: {rmse}")
print(f"R2: {r2}")
print(f"R2: {mae}")

RMSE: 0.2912823031642777
R2: 0.552367140270025
R2: 0.2379800132350321


In [70]:
dt_model.featureImportances

SparseVector(521, {2: 0.0216, 3: 0.0246, 15: 0.008, 16: 0.9436, 424: 0.0003, 462: 0.0019})

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter=10)
gbt_model = gbt.fit(processed_train_df)
gbt_predictions = gbt_model.transform(processed_test_df)

23/03/26 06:31:18 WARN MemoryStore: Not enough space to cache rdd_23_6 in memory! (computed 909.7 MiB so far)
23/03/26 06:31:18 WARN BlockManager: Persisting block rdd_23_6 to disk instead.
23/03/26 06:31:19 WARN MemoryStore: Not enough space to cache rdd_23_2 in memory! (computed 909.7 MiB so far)
23/03/26 06:31:19 WARN BlockManager: Persisting block rdd_23_2 to disk instead.
23/03/26 06:31:19 WARN MemoryStore: Not enough space to cache rdd_23_3 in memory! (computed 909.7 MiB so far)
23/03/26 06:31:19 WARN BlockManager: Persisting block rdd_23_3 to disk instead.
23/03/26 06:31:24 WARN MemoryStore: Not enough space to cache rdd_23_1 in memory! (computed 1364.5 MiB so far)
23/03/26 06:31:24 WARN BlockManager: Persisting block rdd_23_1 to disk instead.
23/03/26 06:31:31 WARN MemoryStore: Not enough space to cache rdd_23_4 in memory! (computed 2046.8 MiB so far)
23/03/26 06:31:31 WARN BlockManager: Persisting block rdd_23_4 to disk instead.
23/03/26 06:31:31 WARN MemoryStore: Not enough s

23/03/26 06:32:23 WARN MemoryStore: Not enough space to cache rdd_23_0 in memory! (computed 6.9 GiB so far)
23/03/26 06:32:23 WARN BlockManager: Persisting block rdd_23_0 to disk instead.


23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_5 in memory! (computed 252.9 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_4 in memory! (computed 2046.8 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_5 in memory! (computed 13.9 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_6 in memory! (computed 168.6 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_1 in memory! (computed 2046.8 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_4 in memory! (computed 9.1 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_0 in memory! (computed 2046.8 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_1 in memory! (computed 5.3 MiB so far)
23/03/26 06:33:33 WARN MemoryStore: Not enough space to cache rdd_23_6 in memory! (computed 9.1 MiB so far)
23/03/26 06:33

23/03/26 06:34:47 WARN MemoryStore: Not enough space to cache rdd_23_9 in memory! (computed 1364.5 MiB so far)
23/03/26 06:34:47 WARN MemoryStore: Not enough space to cache rdd_23_8 in memory! (computed 1364.5 MiB so far)
23/03/26 06:34:47 WARN MemoryStore: Not enough space to cache rdd_23_15 in memory! (computed 589.3 MiB so far)
23/03/26 06:34:47 WARN BlockManager: Persisting block rdd_23_9 to disk instead.
23/03/26 06:34:47 WARN BlockManager: Persisting block rdd_23_15 to disk instead.
23/03/26 06:34:47 WARN MemoryStore: Not enough space to cache rdd_23_10 in memory! (computed 1364.5 MiB so far)
23/03/26 06:34:47 WARN BlockManager: Persisting block rdd_23_10 to disk instead.
23/03/26 06:34:47 WARN BlockManager: Persisting block rdd_23_8 to disk instead.
23/03/26 06:34:47 WARN MemoryStore: Not enough space to cache rdd_23_11 in memory! (computed 1364.5 MiB so far)
23/03/26 06:34:47 WARN BlockManager: Persisting block rdd_23_11 to disk instead.
23/03/26 06:34:48 WARN MemoryStore: Not 

23/03/26 06:35:54 WARN MemoryStore: Not enough space to cache rdd_23_13 in memory! (computed 3.0 GiB so far)
23/03/26 06:35:55 WARN MemoryStore: Not enough space to cache rdd_23_16 in memory! (computed 252.9 MiB so far)
23/03/26 06:35:55 WARN BlockManager: Persisting block rdd_23_16 to disk instead.
23/03/26 06:35:55 WARN MemoryStore: Not enough space to cache rdd_23_12 in memory! (computed 252.9 MiB so far)
23/03/26 06:35:56 WARN MemoryStore: Not enough space to cache rdd_23_12 in memory! (computed 112.4 MiB so far)
23/03/26 06:35:56 WARN MemoryStore: Not enough space to cache rdd_23_13 in memory! (computed 380.1 MiB so far)


...

23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_9 in memory! (computed 1364.5 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_10 in memory! (computed 1364.5 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_13 in memory! (computed 909.7 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_8 in memory! (computed 2046.8 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_14 in memory! (computed 909.7 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_11 in memory! (computed 1364.5 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_15 in memory! (computed 252.9 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_23_12 in memory! (computed 1364.5 MiB so far)
23/03/26 08:29:13 WARN MemoryStore: Not enough space to cache rdd_222_15 in memory! (computed 6.2 MiB so far)

23/03/26 08:29:44 WARN MemoryStore: Not enough space to cache rdd_23_16 in memory! (computed 3.0 GiB so far)
23/03/26 08:29:44 WARN MemoryStore: Not enough space to cache rdd_23_17 in memory! (computed 3.0 GiB so far)
23/03/26 08:29:46 WARN MemoryStore: Not enough space to cache rdd_23_18 in memory! (computed 3.0 GiB so far)
23/03/26 08:29:48 WARN MemoryStore: Not enough space to cache rdd_222_16 in memory! (computed 172.8 MiB so far)
23/03/26 08:29:49 WARN MemoryStore: Not enough space to cache rdd_222_18 in memory! (computed 99.2 MiB so far)


23/03/26 08:30:08 WARN MemoryStore: Not enough space to cache rdd_23_19 in memory! (computed 6.9 GiB so far)


23/03/26 08:30:20 WARN MemoryStore: Not enough space to cache rdd_23_3 in memory! (computed 909.7 MiB so far)
23/03/26 08:30:20 WARN MemoryStore: Not enough space to cache rdd_23_1 in memory! (computed 909.7 MiB so far)
23/03/26 08:30:20 WARN MemoryStore: Not enough space to cache rdd_23_4 in memory! (computed 909.7 MiB so far)
23/03/26 08:30:20 WARN MemoryStore: Not enough space to cache rdd_222_1 in memory! (computed 16.2 MiB so far)
23/03/26 08:30:20 WARN MemoryStore: Not enough space to cache rdd_222_3 in memory! (computed 16.2 MiB so far)
23/03/26 08:30:20 WARN MemoryStore: Not enough space to cache rdd_222_4 in memory! (computed 16.2 MiB so far)
23/03/26 08:30:21 WARN MemoryStore: Not enough space to cache rdd_23_2 in memory! (computed 1364.5 MiB so far)
23/03/26 08:30:21 WARN MemoryStore: Not enough space to cache rdd_222_2 in memory! (computed 1024.6 KiB so far)
23/03/26 08:30:21 WARN MemoryStore: Not enough space to cache rdd_23_6 in memory! (computed 1364.5 MiB so far)
23/03/

23/03/26 08:31:07 WARN MemoryStore: Not enough space to cache rdd_23_9 in memory! (computed 1364.5 MiB so far)
23/03/26 08:31:07 WARN MemoryStore: Not enough space to cache rdd_23_12 in memory! (computed 909.7 MiB so far)
23/03/26 08:31:07 WARN MemoryStore: Not enough space to cache rdd_23_13 in memory! (computed 909.7 MiB so far)
23/03/26 08:31:07 WARN MemoryStore: Not enough space to cache rdd_23_14 in memory! (computed 909.7 MiB so far)
23/03/26 08:31:07 WARN MemoryStore: Not enough space to cache rdd_23_15 in memory! (computed 589.3 MiB so far)
23/03/26 08:31:08 WARN MemoryStore: Not enough space to cache rdd_23_11 in memory! (computed 1364.5 MiB so far)
23/03/26 08:31:08 WARN MemoryStore: Not enough space to cache rdd_23_8 in memory! (computed 2046.8 MiB so far)
23/03/26 08:31:08 WARN MemoryStore: Not enough space to cache rdd_222_9 in memory! (computed 43.2 MiB so far)
23/03/26 08:31:08 WARN MemoryStore: Not enough space to cache rdd_23_10 in memory! (computed 1364.5 MiB so far)


In [15]:
gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)

In [18]:
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

RMSE: 0.27966298405490797
r2: 0.5873672117663772
mae: 0.2276378709295658
